# Day 8: Resonant Collinearity

## Import libraries

In [2]:
import copy
from collections import defaultdict

## Import data

In [3]:
# *** [IMPORT DATA] ***
# NOTE: In the given puzzle input:
# - The grid map represents antennas.
# - EACH antenna is represented by a *letter* or *digit*.
# - The FREQUENCY of EACH antenna is different, based on whether the antenna is a: lowercase letter, uppercase letter or digit.
# - '#': Represents an antinode. 
# =====================================================================================================================
# ! Open the file for reading mode (= default mode if the mode is not specified)
file = open("../data/24_day-8_input-test.txt", "r") 

# Read all the data in the file
file_data = file.read().strip()

# Separate data by line to create rows for grid
grid = file_data.split("\n")

# Separate data in EACH row to represent EACH column
for i in range(len(grid)):
    grid[i] = list(grid[i])

print(grid)
# ====================================================================================================================

[['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.'], ['.', '.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.', '.'], ['.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', 'A', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']]


## Helper functions

In [22]:
def get_antinodes(_grid):
    antennas = []
    numRows = len(_grid)
    numCols = len(_grid[0])
    #antinodes = set()
    antinodes_map = defaultdict(list)
    freq_map = defaultdict(list)

    # Store antennas & their respective locations
    for x in range(numRows):
        for y in range(numCols):
            if _grid[x][y] != '.': # If the grid block is NOT empty
                antennas.append((_grid[x][y], x, y)) # E.g. ('A', 4, 2)

    """ Calculate the positions of antinodes """
    # Group antennas by frequency
    for ant_freq, x, y in antennas:
        freq_map[ant_freq].append((x, y)) # E.g. '('A', [(0,1), (2,3), ...])'

    # Calculate antinodes for EACH frequency
    for ant_freq, positions in freq_map.items():
        print(ant_freq)
        for i in range(len(positions)):
            for j in range(i + 1, len(positions)): # Compare every single antenna against every other antenna of the same frequency
                x1, y1 = positions[i]
                x2, y2 = positions[j]
                print("Point #1:", (x1, y1))
                print("Point #2:", (x2, y2))
                
                # # Calculate the midpoint between the 2 antenna points
                # mid_x = (x1 + x2) / 2
                # mid_y = (y1 + y2) / 2
                
                # Calculate the DISTANCE between antenna point #2 & #1 (same freq.)
                # - NOTE: Removed 'abs' so that distances between 2 or more points in a straight line can be calculated and matched based on line slopes because antinodes can only exist for antennas on the SAME line from L to R or R to L
                dx = x2 - x1 # minus order is IMPORTANT
                dy = y2 - y1
                print("Distance:", (dx, dy))

                # Traverse through EACH non-empty point in the grid to check if it is an antinode of any 1 of the current 2 antennas with same freq.
                for xn in range(numRows):
                    for yn in range(numCols):
                        #if _grid[xn][yn] != '.': # If the grid block is NOT empty -> can't check this because ALL non-freq blocks are ALL empty in initial grid
                        # NOTE: Follow the SAME minus order of calculating the SAME difference in distance (+/-x; +/-y) between different points in a line
                        # - E.g. For a striaght diagonal line going upwards from L to R with 4 points (2 antennas & 2 potential antinodes): xn - x2; x2 - x1; x1 - xn
                        # Distance between antenna #1 (top) and current block
                        dx1 = x1 - xn # minus order is IMPORTANT
                        dy1 = y1 - yn
                        # ----------------
                        # Distance between current block and antenna #2 (bottom)
                        dx2 = xn - x2 # minus order is IMPORTANT
                        dy2 = yn - y2
                        
                        # IFF the distance between the current point & either of the 2 antennas = the distance between the current 2 antennas (dx1/2 == dx && dy1/2 == dy), then ADD the current non-empty point to the list of antinodes
                        if (dx1 == dx and dy1 == dy): # AND if CURRENT position is NOT == CURRENT ant_freq position !!! ADD CHECK HERE !!!
                            print("Matched distance #1:", (dx1, dy1))
                            print("Matched position #1:", (xn, yn))
                            antinodes_map[ant_freq].append((xn, yn))

                        if (dx2 == dx and dy2 == dy): # AND if CURRENT position is NOT == CURRENT ant_freq position !!! ADD CHECK HERE !!!
                            print("Matched distance #2:", (dx2, dy2))
                            print("Matched position #2:", (xn, yn))
                            antinodes_map[ant_freq].append((xn, yn))

                        


                # if dx == 0:
                #     if abs(y2 - y1) == 2 * abs(y1 - mid_y):
                #         antinodes.add((int(mid_x), int(mid_y)))
                # elif dy == 0:
                #     if abs(x2 - x1) == 2 * abs(x1 - mid_x):
                #         antinodes.add((int(mid_x), int(mid_y)))
                # else:
                #     if abs(x2 - x1) == 2 * abs(x1 - mid_x) and abs(y2 - y1) == 2 * abs(y1 - mid_y):
                #         antinodes.add((int(mid_x), int(mid_y)))

    return antinodes_map

## Part 1

In [23]:
# *** [PART 1] ***
# ! PROBLEM: The signal only applies its nefarious effect at specific antinodes based on the resonant frequencies of the antennas.
# - In particular, an *antinode* occurs at *any point* that is perfectly *in line* with TWO antennas of the SAME frequency - but ONLY when ONE of the antennas is TWICE as far away as the other. This means that for any PAIR of antennas with the SAME frequency, there are TWO antinodes, one on EITHER side of them.
# - Antennas with DIFFERENT frequencies DO NOT create antinodes; however, antinodes CAN occur at the SAME locations that CONTAIN antennas.
# - TODO: Calculate the impact of the signal. How many unique locations within the bounds of the map contain an antinode?
# ====================================================================================================================
# ! Create a deep (independent) copy of the grid data, such that changes made to the copy do not affect the original grid to still test/re-run Part 1 with the correct INITIAL (and not modified) grid
# - NOTE: Not using a deep copy will modify the original grid after running Part 1, therefore no correct output will be calculated anymore
part1_grid = copy.deepcopy(grid)

print(get_antinodes(part1_grid))

# ====================================================================================================================

0
Point #1: (1, 8)
Point #2: (2, 5)
Distance: (1, -3)
Matched distance #1: (1, -3)
Matched position #1: (0, 11)
Matched distance #2: (1, -3)
Matched position #2: (3, 2)
Point #1: (1, 8)
Point #2: (3, 7)
Distance: (2, -1)
Matched distance #2: (2, -1)
Matched position #2: (5, 6)
Point #1: (1, 8)
Point #2: (4, 4)
Distance: (3, -4)
Matched distance #2: (3, -4)
Matched position #2: (7, 0)
Point #1: (2, 5)
Point #2: (3, 7)
Distance: (1, 2)
Matched distance #1: (1, 2)
Matched position #1: (1, 3)
Matched distance #2: (1, 2)
Matched position #2: (4, 9)
Point #1: (2, 5)
Point #2: (4, 4)
Distance: (2, -1)
Matched distance #1: (2, -1)
Matched position #1: (0, 6)
Matched distance #2: (2, -1)
Matched position #2: (6, 3)
Point #1: (3, 7)
Point #2: (4, 4)
Distance: (1, -3)
Matched distance #1: (1, -3)
Matched position #1: (2, 10)
Matched distance #2: (1, -3)
Matched position #2: (5, 1)
A
Point #1: (5, 6)
Point #2: (8, 8)
Distance: (3, 2)
Matched distance #1: (3, 2)
Matched position #1: (2, 4)
Matched 

## Part 2

In [ ]:
# *** [PART 2] ***
# ! PROBLEM: xxx
# - TODO: xxx
#====================================================================================================================
# ! Create a deep (independent) copy of the grid data, such that changes made to the copy do not affect the original grid to still test/re-run Part 1 with the correct INITIAL (and not modified) grid
# - NOTE: Not using a deep copy will modify the original grid after running Part 1, therefore no correct output will be calculated anymore
part2_grid = copy.deepcopy(grid)


In [14]:
from collections import defaultdict

word_freq = defaultdict(list)

text = "apple banana apple orange banana banana"

for word in text.split():
    print(word)
    word_freq[word].append(word)

print(word_freq.items())

for word, freq in word_freq.items():
    print(f"{word}: {len(freq)}")

apple
banana
apple
orange
banana
banana
dict_items([('apple', ['apple', 'apple']), ('banana', ['banana', 'banana', 'banana']), ('orange', ['orange'])])
apple: 2
banana: 3
orange: 1
